In [59]:
import pandas as pd

df = pd.read_csv('test.tsv', sep='\t')

In [60]:
print(df['category'].unique())
# count number of each category
print(df['category'].value_counts())

['business' 'health' 'politics' 'sports' 'technology']
category
business      100
health        100
politics      100
sports        100
technology     22
Name: count, dtype: int64


In [61]:
from sklearn.preprocessing import LabelEncoder
clean_df = df.copy()

clean_df.drop('url', axis=1, inplace=True)
le = LabelEncoder()
clean_df['category'] = le.fit_transform(df['category'])

In [62]:
from datasets import Dataset

dataset = Dataset.from_pandas(clean_df)
data = dataset.train_test_split(test_size=0.2, shuffle=True)

In [63]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('almanach/camembert-base')

In [124]:
def tokenize_function(examples):
    return tokenizer(examples["headline"], truncation=True)

In [125]:
tokenized_datasets = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

Map: 100%|██████████| 85/85 [00:00<00:00, 4125.39 examples/s]


In [126]:
tokenized_data["train"][0]

{'category': 2,
 'headline': 'Birame Senghor: "la France a tué mon père en 1944 à Thiaroye"',
 'text': 'Au Sénégal, chaque 1er décembre, on commémore le massacre de Thiaroye. En 1944, plusieurs dizaines de tirailleurs sénégalais ont été tués par l\'armée française. Selon les autorités françaises, il s\'agissait d\'une mutinerie qui avait mal tourné. L\'année 2020 marque les 76 ans de cet évènement tragique. Biram Senghor, fils de Mbapp Senghor, un tirailleur sénégalais mort à Thiaroye, demande toujours justice et réparation à la France pour ce qu’il assimile à un "crime odieux". Par ailleurs, il déclare que la France a une dette de sang vis-à-vis des Africains. Ndèye Khady Lo et Alassane Dia l\'ont rencontré à Dakar.',
 'input_ids': [5,
  17202,
  3188,
  156,
  7345,
  8538,
  92,
  87,
  321,
  184,
  33,
  5930,
  129,
  819,
  22,
  13694,
  15,
  6948,
  55,
  6287,
  35,
  130,
  6,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [134]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][5]["input_ids"])
print(len(tokens))
print(tokens)

tokenizer.model_max_length

17
['<s>', '▁Ramadan', '▁:', '▁Je', 'û', 'ner', '▁en', '▁toute', '▁sécurité', '▁pendant', '▁la', '▁crise', '▁du', '▁cor', 'ona', 'virus', '</s>']


512